In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# STEP 2: Load Dataset
# =========================================
from google.colab import files
uploaded = files.upload()

Saving synthetic_insurance_50k.csv to synthetic_insurance_50k.csv


In [ ]:
# Replace with your file name
df = pd.read_csv("synthetic_insurance_50k.csv")

print("Data Shape:", df.shape)
print(df.head())

Data Shape: (50000, 11)
   Age State Insurance Type  Annual Premium (AUD)  Claim Amount (AUD)  \
0   54   VIC           Life               3226.72             8762.05   
1   38   TAS         Health               4255.91             5699.27   
2   56   QLD         Health               6318.77            12724.00   
3   78   ACT            Car               3404.39             6785.40   
4   23   NSW           Life               1816.43              602.24   

  Claim Status Policy Start Date Policy End Date Product Tier  \
0     Approved        21-02-2023      21-02-2024      Premium   
1     Approved        10-07-2023      09-07-2024      Premium   
2     Approved        23-09-2023      22-09-2024      Premium   
3     Approved        12-08-2024      12-08-2025         Gold   
4     Approved        30-06-2023      29-06-2024        Basic   

  Payment Frequency  Risk Score  
0          Annually        0.72  
1          Annually        0.49  
2          Annually        0.75  
3         

In [ ]:
# STEP 3: Define Features & Target
# =========================================
target_col = "Product Tier"

X = df.drop(target_col, axis=1)
y = df[target_col]

# Encode target (classes → numbers)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# One-hot encode categorical features
X = pd.get_dummies(X)

# Scale numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

Train shape: (40000, 1483) Test shape: (10000, 1483)


In [ ]:
# STEP 4: Handle Class Imbalance (if any)
# =========================================
class_weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(y), y=y
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: np.float64(1.0), 1: np.float64(1.0), 2: np.float64(1.0), 3: np.float64(1.0)}


In [ ]:
# STEP 5: Build Deep ANN Model
# =========================================
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 512)            │       759,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 936,388 (3.57 MB)

 Trainable params: 934,468 (3.56 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    verbose=1
)

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.3995 - loss: 1.2796 - val_accuracy: 0.5980 - val_loss: 0.7314
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.5852 - loss: 0.7761 - val_accuracy: 0.6109 - val_loss: 0.6882
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6081 - loss: 0.7224 - val_accuracy: 0.6284 - val_loss: 0.6524
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.6103 - loss: 0.6984 - val_accuracy: 0.6382 - val_loss: 0.6318
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6290 - loss: 0.6585 - val_accuracy: 0.6338 - val_loss: 0.6130
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6318 - loss: 0.6519 - val_accuracy: 0.6339 - val_loss: 0.6170
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.6441 - loss: 0.6316 - val_accuracy: 0.6248 - val_loss: 0.6076
Epoch 8/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.6480 - loss: 0.6214

In [ ]:
# STEP 7: Evaluate ANN
# =========================================
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6286 - loss: 1.0493
✅ Test Accuracy: 0.64


In [ ]:
sample = X_test[:1]
pred = model.predict(sample)
recommended_tire = label_encoder.inverse_transform([pred.argmax()])
print("🔮 Recommended Tire:", recommended_tire[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
🔮 Recommended Tire: Standard
